<a href="https://colab.research.google.com/github/YooNayoung/ESAA/blob/main/ch9_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **08. 파이썬 추천 시스템 패키지 - Surprise**


In [ ]:
!pip install scikit-surprise

In [3]:
import surprise 

## **Surprise 패키지 소개**
<장점>
- 다양한 추천 알고리즘, 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있다. 
- Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성되었다. 예를 들어 fit(), predict() API로 추천 데이터 학습과 예측, train_test_split()으로 추천 학습 데이터 세트와 예측 데이터 세트 분리, cross_validate(), GridSearchCV 클래스를 통해 추천 시스템을 위한 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능을 제공한다. 

## **Surprise 를 이용한 추천 시스템 구축**

In [4]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [6]:
data = Dataset.load_builtin('ml-100k', prompt=False) 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

In [7]:
algo = SVD()
algo.fit(trainset) 

In [8]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.3540188378034066, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.039458406850425, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.0915226423288065, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6000728986508816, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.280887931791916, details={'was_impossible': False})]

In [9]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.3540188378034066),
 ('882', '291', 4.039458406850425),
 ('535', '507', 4.0915226423288065)]

In [10]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.99   {'was_impossible': False}


In [11]:
accuracy.rmse(predictions)

RMSE: 0.9483


0.9482604598503717

## **Surprise 주요 모듈 소개**

**Dataset**
- Surprise는 user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용 가능. 앞 3개 필드만 로딩하고 이후 필드는 로딩에서 제외됨.
- Dataset.load_builtin : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 내려받기
- Dataset.load_from_file : OS 파일에서 데이터를 로딩할 때 사용
- Dataset.load_from_df : 판다스의 DataFrame에서 데이터 로딩 

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**


In [12]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/drive/MyDrive/ESAA(OB)/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [13]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/drive/MyDrive/ESAA(OB)/ml-latest-small/ratings_noh.csv',reader=reader)

Reader 클래스의 주요 생성 파라미터
- line_format(string): 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식.
- sep(char): 칼럼을 분리하는 분리자이며, 디폴트는 '\t'. 판다스 DataFrame에서 입력받을 경우에는 기재할 필요가 없음.
- rating_scale(tuple,optional): 평점 값의 최소 ~ 최대 평점을 설정. 디폴트는 (1,5)이지만 ratings.csv 파일의 경우는 최소 평점이 0.5, 최대 평점이 5이므로 (0.5,5)로 설정함.

In [14]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [16]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## **Surprise 추천 알고리즘 클래스**
- SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
- KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘.
- BaselineOnly: 사용자 Bias와 아이템 Bias를 감안함 SGD 베이스라인 알고리즘.

## **베이스라인 평점**
- Baseline Rating: 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것 
- 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 
- 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
- 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점 
- 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점 

## **교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝**
- cross_validate()



In [18]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8729  0.8835  0.8709  0.8742  0.8697  0.8742  0.0049  
MAE (testset)     0.6704  0.6793  0.6689  0.6715  0.6711  0.6722  0.0037  
Fit time          4.47    4.39    5.03    4.36    4.46    4.54    0.25    
Test time         0.13    0.28    0.14    0.26    0.14    0.19    0.07    


{'fit_time': (4.470295429229736,
  4.391977071762085,
  5.02804708480835,
  4.361371755599976,
  4.460088491439819),
 'test_mae': array([0.67036119, 0.67934812, 0.66885289, 0.67148672, 0.67105902]),
 'test_rmse': array([0.87289707, 0.88351406, 0.87091164, 0.87422272, 0.86968216]),
 'test_time': (0.13054370880126953,
  0.27887654304504395,
  0.14062905311584473,
  0.2604525089263916,
  0.1375114917755127)}

- GridSearchCV

In [19]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8759573664406867
{'n_epochs': 20, 'n_factors': 50}


## **Surprise 를 이용한 개인화 영화 추천 시스템 구축**

In [21]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ESAA(OB)/ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [22]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [23]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/drive/MyDrive/ESAA(OB)/ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [24]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [25]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [26]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
